In [32]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import openai
import os
from io import StringIO

In [33]:
CredsFile = "/Users/mattraghu/Documents/chatgptsheets-7c47879663f8.json"
SheetID = "1Kr8XX4_lO-W02asMQnaP3PFyS9I3FkD19EjhQhjO4yM"

In [34]:
def setup_api():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(CredsFile, scope)
    client = gspread.authorize(creds)
    return client

In [35]:
def get_sheet(client, sheet_id):
    sheet = client.open_by_key(sheet_id)
    return sheet

In [36]:
client = setup_api()
sheet = get_sheet(client, SheetID)

In [37]:
#Convert the sheet to a string that can be written into a csv File
firstSheet = sheet.get_worksheet(0)
data = firstSheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
string_buffer = StringIO()
df.to_csv(string_buffer, index=False,header=True)
raw_text = string_buffer.getvalue()

In [38]:
# Set up the Open AI API

#Open the api key from my documents
with open(os.path.expanduser("~/Documents/OPENAI_API_KEY.txt")) as f:
    openai.api_key = f.read().strip()
    

#Set up the Organization
openai.organization = "org-9rUnSTHOfxLvlgIoNDRgEDoZ"

In [39]:
# prompt = "Add another column to the table with the name 'Full Name' and set the values to the element names"
prompt = input("Enter a prompt: ")
# Set up the prompt
messages = [
    {"role": "system", "content": "You are a bot used to read spreadsheet data and adjust it based on user input."},
    {"role": "user", "content": "Prompt: " + prompt},
    {"role": "user", "content": "Spreadsheet: " + raw_text},
]

# Set up the completion

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages = messages,
)

outputText = response.choices[0].message.content

In [40]:
cols = outputText.split("\n")
#Now split rows by commas
rows = []
for col in cols:
    rows.append(col.split(","))
rows

[["Here's the spreadsheet with the added columns for element's full name and discovery date:"],
 [''],
 ['atomic number', 'symbol', 'full name', 'discovery date', 'atomic mass'],
 ['1', 'H', 'Hydrogen', '1766', '1.00794'],
 ['2', 'He', 'Helium', '1868', '4.002602'],
 ['3', 'Li', 'Lithium', '1817', '6.941'],
 ['4', 'Be', 'Beryllium', '1798', '9.012182'],
 ['5', 'B', 'Boron', '1808', '10.811'],
 ['6', 'C', 'Carbon', 'Ancient', '12.0107'],
 ['7', 'N', 'Nitrogen', '1772', '14.0067'],
 ['8', 'O', 'Oxygen', '1774', '15.9994'],
 ['9', 'F', 'Fluorine', '1670', '18.9984032'],
 ['10', 'Ne', 'Neon', '1898', '20.1797'],
 ['11', 'Na', 'Sodium', '1807', '22.98976928'],
 ['12', 'Mg', 'Magnesium', '1808', '24.305'],
 ['13', 'Al', 'Aluminum', '1825', '26.9815386'],
 ['14', 'Si', 'Silicon', '1824', '28.0855'],
 ['15', 'P', 'Phosphorus', '1669', '30.973762'],
 ['16', 'S', 'Sulfur', 'Ancient', '32.065'],
 ['17', 'Cl', 'Chlorine', '1774', '35.453'],
 ['18', 'Ar', 'Argon', '1894', '39.948'],
 ['19', 'K', 'P

In [41]:
#Write the output to the google sheet
outputSheet = sheet.get_worksheet(1)
outputSheet.update(rows)

{'spreadsheetId': '1Kr8XX4_lO-W02asMQnaP3PFyS9I3FkD19EjhQhjO4yM',
 'updatedRange': 'Sheet1!A1:E121',
 'updatedRows': 121,
 'updatedColumns': 5,
 'updatedCells': 597}